In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from scipy.sparse import csr_matrix
from itertools import product, chain
import functools 
import operator 
from csv import reader
import regex as re

# Load the data

In [2]:
def features_into_array(path):
    with open(path, 'r') as read_obj:
        csv_reader = reader(read_obj)
        header = next(csv_reader)
        X = list()
        if header != None:
            for row in csv_reader:
                # row variable is a list that represents a row in csv
                X.append(np.array(row[1]))
                
    X = np.array(X) ## dtype might be changed in something more convenient. For now, dtype = "<U1"
    return X

In [3]:
Xtr0 = features_into_array("data/Xtr0.csv")
Ytr0 = np.genfromtxt("data/Ytr0.csv", delimiter=',', skip_header=1)

Xtr1 = features_into_array("data/Xtr1.csv")
Ytr1 = np.genfromtxt("data/Ytr1.csv", delimiter=',', skip_header=1)

Xtr2 = features_into_array("data/Xtr2.csv")
Ytr2 = np.genfromtxt("data/Ytr2.csv", delimiter=',', skip_header=1)

In [4]:
def accuracy(y_true,y_pred, mode='SVM'):
    n = y_true.shape[0]
    if mode == 'SVM':
        predictions = np.ones(n)
        predictions[y_pred < 0] = 0
    else:
        predictions = np.zeros(n)
        predictions[y_pred >= 0.5] = 1
    
    return np.sum(y_true == predictions) / n

In [5]:
Xtr0[0]

'TCCTGTGCACATCTGCACCCCTGTTGTGGCCACAAAATGATCCGGCACCACCCAGTGGGAGACGACAGAGGTGGCAATGGGGTGTCGGCTCTGACGCCTCC'

## Mismatch Spectrum kernel

For a fixed value k (that needs to be tuned), the k-spectrum kernel is defined as : 


\begin{align*}
K(x,x^{\prime}) := \sum_{u \in \mathcal{A}^k} \phi_{u}(x) \phi_{u}(x^{\prime})
\end{align*}

We relax this constraint by authorizing each word of the alphabet to have up to m mismatches.

In [6]:
def all_possible_substrings(k):
    """
    With a k spectrum kernel, let us find all the possible combinations of chars of size k in the sequence x
    This way, we could index them in the sequence x
    """
    char_list = list(['A', 'C','G','T'])
    alphabet_tuples = list(product(char_list,repeat=k))
    alphabet = list()
    for i in alphabet_tuples:
        alphabet.append(functools.reduce(operator.add, (i)))
    return alphabet

In [7]:
def neighbors(word, m):
    """
    This gives neighbors that differ in exactly m places
    """
    
    char_list = list(['A', 'C','G','T'])
    assert(m <= len(word))

    if m == 0:
        return [word]

    r2 = neighbors(word[1:], m-1)
    r = [c + r3 for r3 in r2 for c in char_list if c != word[0]]

    if (m < len(word)):
        r2 = neighbors(word[1:], m)
        r += [word[0] + r3 for r3 in r2]

    return r

def neighbors2(pattern, m):
    """
    This gives neighbors that differ in at most m places.
    """
    return sum([neighbors(pattern, d2) for d2 in range(m + 1)], [])


In [8]:
def all_possible_substrings_mismatch(k,m):
    """
    With a k spectrum kernel, let us find all the possible combinations of chars of size k in the sequence x
    This way, we could index them in the sequence x
    """
    char_list = list(['A', 'C','G','T'])
    alphabet_tuples = list(product(char_list,repeat=k))
    alphabet = list()
    for i in alphabet_tuples:
        word = functools.reduce(operator.add, (i))
        l= [word]+neighbors2(word,m)[1:]
        alphabet.append(l)
    return alphabet

In [9]:
## TODO : a function that computes occurences 
## with overlapping option without calling regex if we have remaining time (lol)

def pre_indexing_by_sequence_mismatch(x, k, m):
    alphabet = all_possible_substrings_mismatch(k, m)
    d = dict()
    for letters in alphabet :
        cnt = 0
        for letter in letters:
            cnt += len(re.findall(letter, x, overlapped=True))
        d[letters[0]] = cnt
    return d

In [10]:
def pre_indexing_mismatch(X, alphabet):
    """
    Transforms an input array into a sparse matrix encoding the number of occurences of each letter of
    the alphabet composed of substrings of size k
    """
    i = 0
    n = X.shape[0]
    D = np.zeros((n,len(alphabet)))
    for x in X:
        d = dict()
        for letters in alphabet :
            cnt = 0
            for letter in letters:
                cnt += len(re.findall(letter, x, overlapped=True))
            d[letters[0]] = cnt
        data = np.array(list(d.items()))
        D[i] = data[:,1]
        i+=1
    D = csr_matrix(D, dtype = int)
    return D

In [11]:
alphabet = all_possible_substrings_mismatch(3,1)
pre_indexing_mismatch(Xtr0[:5],alphabet).toarray()

array([[11, 17, 13,  7, 19, 18, 13, 17, 13, 16, 17, 12, 11, 10, 17,  8,
        16, 19, 19, 16, 20, 26, 21, 14, 14, 20, 19, 15, 12, 18, 19, 11,
        15, 18, 16, 13, 14, 24, 21, 16, 19, 19, 24, 18, 12, 20, 19, 11,
         6, 13, 10, 10, 18, 16, 14, 14, 16, 20, 20, 17,  3, 11, 17,  8],
       [20, 19, 10, 19, 17, 15,  8, 20, 11, 16,  9, 14, 22, 18, 17, 26,
        18, 12, 12, 19, 18, 19, 11, 15, 12,  6,  2, 13, 19, 18, 12, 25,
        10,  9, 11, 14,  9,  9,  3, 15,  5,  2,  2, 13, 18, 16, 10, 19,
        20, 20, 17, 31, 18, 21, 12, 27, 16, 12,  9, 23, 28, 25, 20, 34],
       [36, 24, 26, 26, 24, 15, 10, 19, 27, 14, 16, 17, 26, 15, 14, 18,
        25, 17, 17, 17, 18,  9,  6,  9, 10,  9,  9, 10, 18,  7, 12, 14,
        24, 13, 15, 13, 21,  7, 12, 11, 12, 14, 14, 13, 17, 13, 12, 14,
        28, 14, 16, 16, 14, 11, 10, 11, 16, 13, 13, 12, 17, 12, 17, 11],
       [32, 22, 22, 22, 20, 12, 12, 16, 23, 16, 18, 17, 19, 10, 18, 18,
        24, 14, 16, 13, 15, 13, 11,  8, 16,  7, 13,  9, 12, 1

In [12]:
def mismatch_spectrum_kernel(X_train, X_val, k, mode="train", m=1):
    """
    Computes the spectrum kernels for X_train (n_train x n_train) and X_validation (on the RKHS generated(?) by
    X_train's samples) which is of shape n_validation x n_train
    "test" mode only gives as output the testing kernel
    """
    alphabet = all_possible_substrings_mismatch(k,m)
    
    D_train = pre_indexing_mismatch(X_train,alphabet).toarray()
    D_val = pre_indexing_mismatch(X_val,alphabet).toarray()
    
    K_val = np.inner(D_val, D_train)
    K_val = K_val.astype('float')
    if mode == "test":
        return(K_val)
    else:
        K_train = np.inner(D_train, D_train)
        K_train = K_train.astype('float')
        
        return(K_train, K_val)


In [14]:
## example 

mismatch_spectrum_kernel(Xtr0[:5], Xtr0[16:20], 3,mode='test',m=1)

array([[16158., 14442., 15070., 15164., 15408.],
       [15926., 13732., 15764., 15894., 15986.],
       [15814., 15190., 15012., 14900., 15256.],
       [14978., 16138., 15872., 15384., 15206.]])